## Загрузим нужные библиотеки

In [ ]:


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
from sklearn.metrics import f1_score, make_scorer
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import imblearn

import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV

 

f1 = make_scorer(f1_score , average='macro')

#это дата старта подачи документов
#считаем возраст в момент поступления от этой даты
inDay = 20
inMonth = 6



def fit_plot_confusion(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train )

    mean, std = clf.cv_results_['mean_test_score'][clf.best_index_], \
                clf.cv_results_['std_test_score'][clf.best_index_]

    logging.info(clf.best_params_)

    disp = metrics.plot_confusion_matrix(clf, X_test, y_test , normalize='true')
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()
    
    return clf.best_estimator_, {"mean": mean, "std": std}



from sklearn.model_selection import RandomizedSearchCV


%matplotlib inline

from datetime import date

def calculate_age(born):
    f = "%Y-%m-%d 00:00:00.000"
    born = datetime.strptime(born, f)
    today = date.today()
    return today.year - born.year - ((inMonth, inDay) < (born.month, born.day))

def calculate_monBr(born):
    f = "%Y-%m-%d 00:00:00.000"
    born = datetime.strptime(born, f)
    return born.month


In [ ]:
#карта численности в тыс. чел. / примерно дальности от Барнаула в км(данные для Москвы и Китая скращены) 
#все данные взяты из открытых истоничков
pop_map = {'Барнаул':[600,0],
           'Бийск':[200,159],
           'Тальменка':[19,85],
           'ЗАТО Сибирский':[15,60],
           'Поспелиха':[12,200],
           'Благовещенка':[11,266],
           'Волчиха':[11,273],
           'Алтайское':[14,245],
           'Кулунда':[14.4,335],
           'Павловск':[14.5,60],
           'Белокуриха':[14.5,226],
           'Яровое':[18.1,444],
           'Москва':[2000,3500],
           'Рубцовск':[147,295],
           'Томск':[1068,490],
           'Заринск':[47.5,115],
           'Камень-на-Оби':[42,169],
           'Славгород':[31,435],
           'Алейск':[28.5,127],
           'Горняк':[13.2,305],
           'Шипуново':[12.7,160],
           'Змеиногорск':[10.6,266],
           'Михайловское':[11,322],
           'Поспелиха':[12,200],
           'Новосибирск':[1600,220],
           'Горно-Алтайск':[65,260],
           'Китай':[2000,3000],
           'Семей':[350,407],
           'Усть-Каменогорск':[316,533],
           'Павлодар':[361,520],
           'Шемонаиха':[18,413],
           'Риддер':[56,566],
           'Душанбе':[829,2800],
           'Кемерово':[548, 412],
           'Мамонтово':[9,183],
           'Топчиха':[9,106],
                      
          }


def getCityPopulation(cityName):
    if cityName not in pop_map:
        return 0
    else:
        return pop_map[cityName][0]/2000.0

def getCityDist(cityName):
    if cityName not in pop_map:
        return 0
    else:
        return pop_map[cityName][1]/5000.0    


In [ ]:
def fix_1000_ball(row):
    ball = row['СрБаллАттестата']
    if  ball > 2500 and ball < 5500:
        ball = ball / 1000
    return ball

def fix_china_ball(row):
    ball = row['СрБаллАттестата']
    parCnt = row['Страна_Родители']
    if parCnt == 'Китай' and  ball < 51:
        ball = ball / 10

    return ball

def fix_low_ball(row):
    ball = row['СрБаллАттестата']

    if  ball < 5.1:
        ball = ball - 3
        ball = ball / 2
        ball = ball * 70
        ball = ball + 30
            
    return ball

In [ ]:
df = pd.read_csv("data/train_dataset_train.csv")

df.loc[df['Год_Поступления'] == 2212, 'Год_Поступления'] = 2012


df.loc[df['Статус'] == 4, 'Статус'] = 2
df.loc[df['Статус'] == 3, 'Статус'] = 0
df.loc[df['Статус'] == -1, 'Статус'] = 1


def replaceInRow(df_m, ColName, search, replace):
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find(search) == -1 else  replace, axis=1)

def replaceInRowSimple(df_m, ColName, search, replace):
    df_m[ColName] = df_m.apply(lambda row: str(row[ColName]).replace(search, replace) , axis=1)
    
def topoFixCountry(df_m, ColName):    
    replaceInRow(df_m, ColName, 'РОССИЯ', 'Россия')
    replaceInRow(df_m, ColName, 'азах', 'Казахстан')
    replaceInRow(df_m, ColName, 'аджи', 'Таджикистан')
    replaceInRow(df_m, ColName, 'ырг', 'Киргизия')
    replaceInRow(df_m, ColName, 'Росссия', 'Россия')
    replaceInRow(df_m, ColName, 'КНР', 'Китай')

    
def topoFixAditional(df_m, ColName):
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("осква") == -1 else  "Москва г", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Семей") == -1 else  "Семей", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Старая Белокуриха") == -1 else  "Старобелокуриха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Старобелокуриха") == -1 else  "Старобелокуриха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Белокуриха") == -1 else  "Белокуриха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Кулунда") == -1 else  "Кулунда", axis=1)

    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Яровое") == -1 else  "Яровое", axis=1)    
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Шемонаиха") == -1 else  "Шемонаиха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("ЗАТО Сибирский") == -1 else  "ЗАТО Сибирский", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Сибирский ЗАТО") == -1 else  "ЗАТО Сибирский", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Павлодар") == -1 else  "Павлодар", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Мамонтово") == -1 else  "Мамонтово", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Усть-Каменогорск") == -1 else  "Усть-Каменогорск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Зудилово") == -1 else  "Зудилово", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Волчиха") == -1 else  "Волчиха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Троицкое") == -1 else  "Троицкое", axis=1)
    
def topoFixAditional2(df_m, ColName):    
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Душанбе") == -1 else  "Душанбе", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Поспелиха") == -1 else  "Поспелиха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Риддер") == -1 else  "Риддер", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Черемное") == -1 else  "Черемное", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Завьялово") == -1 else  "Завьялово", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Смоленское") == -1 else  "Смоленское", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Стрежевой") == -1 else  "Стрежевой", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Кривошеино") == -1 else  "Кривошеино", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Томск") == -1 else  "Томск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Топчиха") == -1 else  "Топчиха", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Новые Зори") == -1 else  "Новые Зори", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Целинное") == -1 else  "Целинное", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Белово") == -1 else  "Белово", axis=1)

def topoFixAditional3(df_m, ColName):    
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Осинники") == -1 else  "Осинники", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Новокузнецк") == -1 else  "Новокузнецк", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Усть-Калманка") == -1 else  "Усть-Калманка", axis=1)
    
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("староалейское") == -1 else  "староалейское", axis=1)
    
    replaceInRowSimple(df_m, ColName, 'Алейский', 'Алский')
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Алейск") == -1 else  "Алейск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Санкт-Петербург") == -1 else  "Санкт-Петербург", axis=1)
    replaceInRowSimple(df_m, ColName, 'Новосибирская', 'Н-ская')
    replaceInRowSimple(df_m, ColName, 'Новосибирской', 'Н-ской')
    replaceInRowSimple(df_m, ColName, 'Новосибирский', 'Н-ский')
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Новосибирск") == -1 else  "Новосибирск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Кемерово") == -1 else  "Кемерово", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Горняк") == -1 else  "Горняк", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Залесово") == -1 else  "Залесово", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Тальменка") == -1 else  "Тальменка", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Барнал") == -1 else  "Барнаул", axis=1)
    
def topoFixAditional4(df_m, ColName):    
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Красногорское") == -1 else  "Красногорское", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Алтайское") == -1 else  "Алтайское", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Родино") == -1 else  "Родино", axis=1)
    replaceInRowSimple(df_m, ColName, 'Павловский', 'П-ский')
    replaceInRowSimple(df_m, ColName, 'Петропавловский', 'ПП-ский')
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Петропавловское") == -1 else  "ПП-ское", axis=1)
    replaceInRowSimple(df_m, ColName, 'Петропавловск-Камчатский', 'ПП-Камчатский')
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Павловск") == -1 else  "Павловск", axis=1)
    
    
def topoFix(df_m, ColName):
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Китай") == -1 else  "Китай", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("КНР") == -1 else  "Китай", axis=1)

    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("арнаул") == -1 else  "Барнаул", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("убцовск") == -1 else  "Рубцовск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("овоалтайск") == -1 else  "Барнаул", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Бийск") == -1 else  "Бийск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Славгород") == -1 else  "Славгород", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Заринск") == -1 else  "Заринск", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("Камень") == -1 else  "Камень-на-Оби", axis=1)
    df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("орно-") == -1 else  "Горно-Алтайск", axis=1)
    
def topoFixExtra(df_m, ColName):    
    topoFix(df_m, ColName)
    topoFixAditional(df_m, ColName)
    topoFixAditional2(df_m, ColName)
    topoFixAditional3(df_m, ColName)
    topoFixAditional4(df_m, ColName)
    
    #эту группировку отключил
    if False:
        df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("с.") == -1 else  "Село", axis=1)

        df_m[ColName] = df_m.apply(lambda row: "АлтСело" if row[ColName] == 'Село'and row['Регион_ПП'] == 'Алтайский край' else row[ColName],  axis=1)
        df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("г.") == -1 else  "Город", axis=1)

        df_m[ColName] = df_m.apply(lambda row: "АлтГород" if row[ColName] == 'Город'and row['Регион_ПП'] == 'Алтайский край' else row[ColName],  axis=1)

        df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("р-н") == -1 else  "СелоРайон", axis=1)
        df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("район") == -1 else  "СелоРайон", axis=1)

        df_m[ColName] = df_m.apply(lambda row: "АлтСелоРайон" if row[ColName] == 'СелоРайон'and row['Регион_ПП'] == 'Алтайский край' else row[ColName],  axis=1)


        if True:        
            df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find(" с") == -1 else  "Село", axis=1)
            df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find("с ") == -1 else  "Село", axis=1)
            df_m[ColName] = df_m.apply(lambda row: row[ColName] if str(row[ColName]).find(" г") == -1 else  "Город", axis=1)



def fixValues(df_m):
    df_m['Город_ПП'].fillna('', inplace=True)
    df_m['Где_Находится_УЗ'].fillna('', inplace=True)
     
    
    df_m['_Город_ПП'] = df_m['Город_ПП']
    df_m['_Где_Находится_УЗ'] = df_m['Где_Находится_УЗ']
    
    df_m['Село'].fillna(0.0, inplace=True)
    df_m['Село'] = df_m['Село'].astype('int64')
    df_m['Иностранец'].fillna(0.0, inplace=True)
    df_m['Иностранец'] = df_m['Иностранец'].astype('int64')
    df_m['КодФакультета'].fillna(0.0, inplace=True)
    df_m['КодФакультета'] = df_m['КодФакультета'].astype('int64')
    df_m['Общежитие'].fillna(0.0, inplace=True)
    df_m['Общежитие'] = df_m['Общежитие'].astype('int64')    
    
    topoFixCountry(df_m, 'Страна_ПП')
    topoFixCountry(df_m, 'Страна_Родители')
    
    
    
    df_m.loc[ (df_m['СрБаллАттестата'] > 5)& (df_m['СрБаллАттестата'] < 30), 'СрБаллАттестата']=60
    df_m.loc[ (df_m['СрБаллАттестата'] < 1), 'СрБаллАттестата']=60
    df_m.loc[ (df_m['СрБаллАттестата'] >200 ) &(df_m['СрБаллАттестата'] <3000 ), 'СрБаллАттестата' ]=60
    df_m.loc[ (df_m['СрБаллАттестата'] < 12), 'oldSch'] =1
    df_m.loc[ (df_m['СрБаллАттестата'] >= 12), 'oldSch'] =0

    df_m['СрБаллАттестата'] = df_m.apply(lambda row: fix_china_ball(row), axis=1)
    df_m['СрБаллАттестата'] = df_m.apply(lambda row: fix_1000_ball(row), axis=1)
    df_m['СрБаллАттестата'] = df_m.apply(lambda row: fix_low_ball(row), axis=1)    

    agu_name = 'АГУ'
    polith_name = 'АГТУ'
    
    df_m['Уч_Заведение'].fillna('', inplace=True)
 
    replaceInRow(df_m, 'Уч_Заведение', 'ФГБОУ ВПО Алтайский государственный университет', agu_name)
    replaceInRow(df_m, 'Уч_Заведение', 'Ползу', polith_name)
    replaceInRowSimple(df_m, 'Уч_Заведение', 'Горно-Алтайский', 'Г-А')
    replaceInRow(df_m, 'Уч_Заведение', 'Алтайский государственный университет', agu_name)

    replaceInRow(df_m, 'Уч_Заведение', 'СОШ', 'СОШ')
    replaceInRow(df_m, 'Уч_Заведение', 'Гимназия', 'Гимназия')
    replaceInRow(df_m, 'Уч_Заведение', 'ицей', 'Лицей')

    replaceInRow(df_m, 'Уч_Заведение', 'средняя общеобразовательная школа', 'СОШ')
    replaceInRow(df_m, 'Уч_Заведение', 'средняя школа', 'СОШ')
    
    #такая группировка тоже не дала большого эффекта
    if False:    
    
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайский государственный технический университет', polith_name)
        replaceInRow(df_m, 'Уч_Заведение', 'Средняя общеобразовательная школа', 'СОШ')
        replaceInRow(df_m, 'Уч_Заведение', 'Г-А государственный университет', 'Г-А-ГУ')
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайский промышленно-экономический колледж', 'Алтайский_промышленно-экономический_колледж')
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайская академия экономики и права', 'Алтайская_академия_экономики_и_права')
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайский государственный педагогический университет', 'Алтайский_государственный_педагогический_университет')
        replaceInRow(df_m, 'Уч_Заведение', 'Барнаульский государственный педагогический университет', 'Алтайский_государственный_педагогический_университет')
        replaceInRow(df_m, 'Уч_Заведение', 'Барнаульский государственный педагогический колледж', 
                     'Барнаульский_государственный_педагогический_колледж')
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайский государственный аграрный университет', 
                     'Алтайский_государственный_аграрный_университет')
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайская академия гостеприимства', 
                     'Алтайская_академия_гостеприимства')
        replaceInRow(df_m, 'Уч_Заведение', 'Российская академия народного хозяйства', 
                     'Российская_академия_народного хозяйства')    
        replaceInRow(df_m, 'Уч_Заведение', 'Алтайская государственная педагогическая академия', 
                     'Алтайская_государственная_педагогическая_академия')   

        replaceInRow(df_m, 'Уч_Заведение', 'Бийский государственный колледж', 
                     'Бийский_государственный_колледж')   

        replaceInRow(df_m, 'Уч_Заведение', 'Алтайский государственный медицинский университет', 
                     'а_г_Мед_у')   
        replaceInRow(df_m, 'Уч_Заведение', 'крайпотребсоюза', 
                     'техникум_крайпотребсоюза')   
        replaceInRow(df_m, 'Уч_Заведение', 'Восточно-Казахстанский государственный университет', 
                     'Восточно-Казахстанский государственный университет')

        replaceInRow(df_m, 'Уч_Заведение', 'Барнаульский торгово-экономический колледж', 
                     'Барнаульский торгово-экономический колледж')    
    

   
    df_m['Регион_ПП'].fillna('', inplace=True)
    df_m['Иностранец'].fillna(0.0, inplace=True)
    replaceInRowSimple(df_m, 'Регион_ПП', 'область', 'обл')
    replaceInRowSimple(df_m, 'Регион_ПП', 'обл.', 'обл')


    replaceInRow(df_m, 'Регион_ПП', 'Алтайский ', 'Алтайский край')
    replaceInRow(df_m, 'Регион_ПП', 'Алайский край', 'Алтайский край')
    
    
    topoFix(df_m, 'Город_ПП')
    topoFix(df_m, 'Где_Находится_УЗ')
   

    topoFixExtra(df_m, '_Город_ПП')
    topoFixExtra(df_m, '_Где_Находится_УЗ')

    df_m['gor_pop'] = df_m.apply(lambda row: getCityPopulation(row['_Город_ПП']), axis=1)
    df_m['us_pop'] = df_m.apply(lambda row: getCityPopulation(row['_Где_Находится_УЗ']), axis=1)
    df_m['gor_dst'] = df_m.apply(lambda row: getCityDist(row['_Город_ПП']), axis=1)
    df_m['us_dst'] = df_m.apply(lambda row: getCityDist(row['_Где_Находится_УЗ']), axis=1)
    
    df_m.loc[df_m['Изучаемый_Язык'] == 'Русский язык', 'Изучаемый_Язык'] = 'Английский язык'
    
    df_m['Изучаемый_Язык'].fillna('', inplace=True)
    
    df_m['Изучаемый_Язык'] = df_m.apply(lambda row: row['Изучаемый_Язык'] if str(row['Изучаемый_Язык']).find("Англ") == -1 else  "Английский язык", axis=1)
    df_m['Изучаемый_Язык'] = df_m.apply(lambda row: row['Изучаемый_Язык'] if str(row['Изучаемый_Язык']).find("Нем") == -1 else  "Немецкий язык", axis=1)
    
    return df_m
    
   

In [ ]:
df = fixValues(df)


In [ ]:
#формирование карт индексов для строквых значения после группировки

gl_map = {}

col_name ='Страна_Родители' 
gl_map[col_name] = df[col_name].unique().tolist()

col_name ='Изучаемый_Язык' 
gl_map[col_name] = df[col_name].unique().tolist()

col_name ='Страна_ПП' 
gl_map[col_name] = df[col_name].unique().tolist()

col_name ='Где_Находится_УЗ' 
gl_map[col_name] = df[col_name].unique().tolist()

col_name ='Город_ПП' 
gl_map[col_name] = df[col_name].unique().tolist()

col_name ='Уч_Заведение' 
gl_map[col_name] = df[col_name].unique().tolist()

col_name ='Регион_ПП' 
gl_map[col_name] = df[col_name].unique().tolist()


    
def getVusIndex(row, colName):
    valueCell = row[colName]
    if valueCell not in gl_map[colName]:
        return -1
    else:
        return gl_map[colName].index(valueCell)


In [ ]:

def calculate_ageIn(born):
    bd = born['Дата_Рождения']
    
    f = "%Y-%m-%d 00:00:00.000"
    
    inY = born['Год_Поступления']
    if inY < 1900:
        return 17
    today = date.today()
    yearsAgoIN = today.year - inY
    born = datetime.strptime(bd, f)
    
    
    age = today.year - born.year - ((inMonth, inDay) < (born.month, born.day))
    ageOnIn = age - yearsAgoIN
    if ageOnIn < 8:
        return 18
    return ageOnIn

def calculate_InOld(row):
    today = date.today()
    if row['Год_Окончания_УЗ'] < 10 or  row['Год_Поступления'] < 10:
        return 0
    return today.year - row['Год_Поступления']

def calculate_gap(row):
    
    if row['Год_Окончания_УЗ'] < 10 or  row['Год_Поступления'] < 10:
        return 0
    return row['Год_Поступления'] - row['Год_Окончания_УЗ']

def form_mark_2(val):
    return (val-30)/70



In [ ]:
df['wom_int'] = df.apply(lambda row: (row['Пол'] == 'Жен')*1.0, axis=1)
gp = df.groupby(['Код_группы']).agg({'wom_int': ['mean']}).reset_index()
m_map = {}
for index, row in gp.iterrows():
    m_map[int(row['Код_группы'])] = float(row['wom_int'])


df['ball4'] = df.apply(lambda row: form_mark_2(row['СрБаллАттестата']), axis=1)
gp = df.groupby(['КодФакультета']).agg({'ball4': ['mean']}).reset_index()
b_map = {}
for index, row in gp.iterrows():
    b_map[int(row['КодФакультета'])] = float(row['ball4'])
    

#--------------------------------
gp = df.groupby(['КодФакультета']).agg({'wom_int': ['mean']}).reset_index()
kaf_pol_map = {}
for index, row in gp.iterrows():
    kaf_pol_map[int(row['КодФакультета'])] = float(row['wom_int'])
    


In [ ]:

def fillFields(df_i):
    #df_i['man_int'] = df_i.apply(lambda row: (row['Пол'] == 'Муж')*1, axis=1)
    df_i['wom_int'] = df_i.apply(lambda row: (row['Пол'] == 'Жен')*1, axis=1)
    df_i['lang_int'] = df_i.apply(lambda row: getVusIndex(row,'Изучаемый_Язык'), axis=1)
    df_i['prCt_int'] = df_i.apply(lambda row: getVusIndex(row,'Страна_Родители'), axis=1)

    df_i['ctPP_int'] = df_i.apply(lambda row: getVusIndex(row,'Страна_ПП'), axis=1)
    df_i['US_location_in'] = df_i.apply(lambda row: getVusIndex(row,'Где_Находится_УЗ'), axis=1)

    df_i['reg_pp_in'] = df_i.apply(lambda row: getVusIndex(row,'Регион_ПП'), axis=1)
    df_i['gorod_pp_in'] = df_i.apply(lambda row: getVusIndex(row,'Город_ПП'), axis=1)
    df_i['uch_zav_indx'] = df_i.apply(lambda row: getVusIndex(row,'Уч_Заведение'), axis=1)
    

    df_i['oldIn'] = df_i.apply(lambda row: calculate_InOld(row), axis=1)
    df_i['m_pol'] = df_i.apply(lambda row: 0.5 if int(row['Код_группы']) not in m_map else  m_map[int(row['Код_группы'])], axis=1)

    df_i['gap'] = df_i.apply(lambda row: calculate_gap(row), axis=1)

    df_i['os_oo'] = df_i.apply(lambda row: (row['Основания'] == 'ОО')*1, axis=1)
    df_i['os_cn'] = df_i.apply(lambda row: (row['Основания'] == 'ЦН')*1, axis=1)
    df_i['os_dn'] = df_i.apply(lambda row: (row['Основания'] == 'ДН')*1, axis=1)
    df_i['os_bn'] = df_i.apply(lambda row: (row['Основания'] == 'БН')*1, axis=1)
    df_i['os_sn'] = df_i.apply(lambda row: (row['Основания'] == 'СН')*1, axis=1)
    df_i['age'] = df_i.apply(lambda row: calculate_age(row['Дата_Рождения']), axis=1)
    df_i['ageIn'] = df_i.apply(lambda row: calculate_ageIn(row), axis=1)
    df_i['ball4'] = df_i.apply(lambda row: form_mark_2(row['СрБаллАттестата']), axis=1)

    df_i['b_pol'] = df_i.apply(lambda row: b_map[int(row['КодФакультета'])], axis=1)

    df_i['kaf_pol_mean'] = df_i.apply(lambda row: kaf_pol_map[int(row['КодФакультета'])], axis=1)
    
    df_i = df_i.fillna(0)
    return df_i

df =fillFields(df)



In [ ]:

fld_list = [ 'Общежитие', 
            'Наличие_Матери', 
            'КодФакультета', 
            #'Год_Поступления',
            "wom_int", #женский пол
            'ball4', #единый бал аттестата
            'uch_zav_indx', #индекс учебного заведения

            'US_location_in',
            'reg_pp_in',
            'gorod_pp_in',
            'oldIn',
           
            'gap',
            'age',
            'Иностранец', 
            'os_dn', 
            'os_bn', 
            'Село',
            'lang_int', 
            'os_oo',
            'os_cn', 
            'os_sn', 
            'b_pol',
            'prCt_int', 
            'ctPP_int',
            'm_pol',
            
            'kaf_pol_mean',
            'Код_группы',
            'oldSch', # если у поступающего был аттестат с  оценками до 5 баллов
            'gor_pop',
            'us_pop',
            'gor_dst',
            'us_dst'
           ]


In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='all', random_state=7, k_neighbors=5) #minority

X = df[fld_list]
y = df[["Статус"]]

#выделение тестовой и обучающей частей
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
#оверсемплинг
X_train, y_train = sm.fit_resample(X_train, y_train)


In [ ]:
#отображение точности
def baseForModel(model):
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    print(f1_score(y_test, pred, average='macro', zero_division = 0))
    disp = metrics.plot_confusion_matrix(model, X_test, y_test , normalize='true')
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

In [ ]:
#
#сама модель!!
#
gsearch = xgb.XGBClassifier( learning_rate=0.1, n_estimators=200, max_depth=6,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, seed=27)

baseForModel(gsearch)

# Cоздание файла с ответом

In [ ]:
df_test = pd.read_csv("data/test_dataset_test.csv", index_col=False)
df_submission = pd.read_csv("data/submission_baseline.csv", index_col=False)

#готовим поля аналогично обучающему датасету
df_test = fixValues(df_test)

df_test = fillFields(df_test)

df_test = df_test[fld_list]


df_test_pred = gsearch.predict(df_test)
df_submission["Статус"] = df_test_pred

df_submission.loc[df_submission['Статус'] == 2, 'Статус'] = 4
df_submission.loc[df_submission['Статус'] == 0, 'Статус'] = 3
df_submission.loc[df_submission['Статус'] == 1, 'Статус'] = -1


df_submission.head(4)

In [ ]:
df_submission.to_csv("sub/submission_8_800.csv", index=False)#13 для тестирования